# Teste pyomo

In [1]:
import numpy as np
from udgp import Instance
from udgp import M2

In [2]:
instance = Instance.artificial_molecule(5, freq=True, seed=123456)
instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
model = M2(instance, nx=4, ny=1)

In [10]:
model.optimize(solver="gurobi_direct", log=True)

        0.05 seconds required for presolve
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.05
Set parameter IntFeasTol to value 0.0001
Set parameter FeasibilityTol to value 0.0001
Set parameter OptimalityTol to value 0.0001
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 445 rows, 252 columns and 1178 nonzeros
Model fingerprint: 0xb7ebc25e
Model has 10 quadratic constraints
Variable types: 202 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 4e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 82 rows and 22 columns
Presolve time: 0.01s
Presolved: 493 rows, 270 columns, 1564 nonzeros
Presolved model has 40 bilinear constraint(s)
Var

True

In [11]:
print(instance.is_solved())
instance.view()

True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol